In [ ]:
import torch

In [ ]:
#思路很简单，就是用文本CLS分类，然后注意一下，这里每次进4个都会用，所以都没mask的事
class bert_model(torch.nn.Module):
    def __init__(self, config, model, loss):
        super().__init__()   
        self.config = config
        self.model = model    
        self.FNN = torch.nn.Linear(config.hidden_size, 1)                              #二分类
        self.dropout = torch.nn.Dropout(0.1)
        self.loss = loss                                                               
        
    def forward(self,input_id,input_mask, tags=None):
        
        bert_output= self.model(input_ids=input_id, attention_mask=input_mask,)   
        CLS = bert_output[0][:,0,:]
        logits = torch.sigmoid(self.FNN(self.dropout(CLS)))                            #结果
        
        if tags==None:
            output = (logits,)
        else:
            new_logits = logits.view(-1,logits.shape[-1]).squeeze(1).float()
            new_tags = tags.view(-1).float()
            loss = self.loss(new_logits,new_tags)                                      #不需要考虑mask，因为每组数据都用上了
            output = (logits,) + (loss,)
        return output       